#### Importando pacotes necessários

In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [2]:
import warnings
warnings.filterwarnings('ignore')

#### Upload das bases de dados

In [3]:
listings = pd.read_csv("listings.csv", sep=",")
#calendar = pd.read_csv("calendar.csv", sep=",")
#reviews = pd.read_csv("reviews.csv", sep=",")

In [4]:
listings.head(3)

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,17878,https://www.airbnb.com/rooms/17878,20210222161029,2021-02-23,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,https://a0.muscache.com/pictures/65320518/3069...,68997,https://www.airbnb.com/users/show/68997,...,10.0,10.0,9.0,NaN,t,1,1,0,0,2.01
1,24480,https://www.airbnb.com/rooms/24480,20210222161029,2021-02-23,Nice and cozy near Ipanema Beach,My studio is located in the best of Ipanema. ...,"The beach, the lagoon, Ipanema is a great loca...",https://a0.muscache.com/pictures/11955612/b28e...,99249,https://www.airbnb.com/users/show/99249,...,10.0,10.0,9.0,NaN,f,1,1,0,0,0.67
2,25026,https://www.airbnb.com/rooms/25026,20210222161029,2021-02-23,Beautiful Modern Decorated Studio in Copa,"Our apartment is a little gem, everyone loves ...",Copacabana is a lively neighborhood and the ap...,https://a0.muscache.com/pictures/3003965/68ebb...,3746246,https://www.airbnb.com/users/show/3746246,...,10.0,10.0,9.0,NaN,f,11,11,0,0,1.82


In [5]:
df1 = listings

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26615 entries, 0 to 26614
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            26615 non-null  int64  
 1   listing_url                                   26615 non-null  object 
 2   scrape_id                                     26615 non-null  int64  
 3   last_scraped                                  26615 non-null  object 
 4   name                                          26586 non-null  object 
 5   description                                   25160 non-null  object 
 6   neighborhood_overview                         14314 non-null  object 
 7   picture_url                                   26615 non-null  object 
 8   host_id                                       26615 non-null  int64  
 9   host_url                                      26615 non-null 

#### Criando variável last_scraped_n
- Volume de dias da data de carregamento destas bases e ultipo scraping no anúncio
- Utilizando a variável last_scraped para o cálculo

In [7]:
# Criando variável em dias
df1["last_scraped"] = df1["last_scraped"].astype("string")
df1["last_scraped"] = pd.to_datetime(df1["last_scraped"])
df1["last_day"] = "2021-02-27"
df1["last_day"] = pd.to_datetime(df1["last_day"])
df1["last_scraped_d"] = df1["last_day"] - df1["last_scraped"]
df1["last_scraped_d"] = df1["last_scraped_d"] / np.timedelta64(1, "D")
print(pd.value_counts(df1["last_scraped_d"]))

3.0    13192
4.0    12205
2.0      870
5.0      348
Name: last_scraped_d, dtype: int64


In [8]:
# Criando variável em meses
df1["last_scraped"] = df1["last_scraped"].astype("string")
df1["last_scraped"] = pd.to_datetime(df1["last_scraped"])
df1["last_day"] = "2021-02-27"
df1["last_day"] = pd.to_datetime(df1["last_day"])
df1["last_scraped_m"] = df1["last_day"] - df1["last_scraped"]
df1["last_scraped_m"] = df1["last_scraped_m"] / np.timedelta64(1, "M")
print(pd.value_counts(df1["last_scraped_m"]))

0.098565    13192
0.131420    12205
0.065710      870
0.164274      348
Name: last_scraped_m, dtype: int64


In [9]:
# Criando variável em anos
df1["last_scraped"] = df1["last_scraped"].astype("string")
df1["last_scraped"] = pd.to_datetime(df1["last_scraped"])
df1["last_day"] = "2021-02-27"
df1["last_day"] = pd.to_datetime(df1["last_day"])
df1["last_scraped_y"] = df1["last_day"] - df1["last_scraped"]
df1["last_scraped_y"] = df1["last_scraped_y"] / np.timedelta64(1, "Y")
print(pd.value_counts(df1["last_scraped_y"]))

0.008214    13192
0.010952    12205
0.005476      870
0.013690      348
Name: last_scraped_y, dtype: int64


#### Criando variável com tipo da imagem da url
- Usando a coluna picture_url
- Motivo: os diferentes tipos de imagens carregadas podem ser significantes

In [10]:
pd.set_option("max_colwidth", None)
df1["picture_url"].head(5)

0    https://a0.muscache.com/pictures/65320518/30698f38_original.jpg
1    https://a0.muscache.com/pictures/11955612/b28e9db0_original.jpg
2     https://a0.muscache.com/pictures/3003965/68ebb634_original.jpg
3    https://a0.muscache.com/pictures/20009355/38b6a10a_original.jpg
4    https://a0.muscache.com/pictures/23782972/1d3e55b0_original.jpg
Name: picture_url, dtype: object

In [11]:
picture_url = pd.DataFrame(df1.picture_url.str.split(".", expand = True))
print(picture_url.head(3))

            0         1                                        2    3
0  https://a0  muscache  com/pictures/65320518/30698f38_original  jpg
1  https://a0  muscache  com/pictures/11955612/b28e9db0_original  jpg
2  https://a0  muscache   com/pictures/3003965/68ebb634_original  jpg


In [12]:
picture_url = pd.DataFrame({"picture_url_tipo": picture_url.iloc[:,3]})
print(picture_url.head(3))

  picture_url_tipo
0              jpg
1              jpg
2              jpg


In [13]:
print(pd.value_counts(picture_url["picture_url_tipo"]))

jpg     25660
jpeg      855
JPEG       54
png        43
Name: picture_url_tipo, dtype: int64


In [14]:
picture_url["picture_url_tipo"] = picture_url["picture_url_tipo"].str.lower()
print(pd.value_counts(picture_url["picture_url_tipo"]))

jpg     25660
jpeg      909
png        43
Name: picture_url_tipo, dtype: int64


In [15]:
df1['picture_url_tipo'] = picture_url["picture_url_tipo"]
print(pd.value_counts(picture_url["picture_url_tipo"]))

jpg     25660
jpeg      909
png        43
Name: picture_url_tipo, dtype: int64


#### Calculando o tempo de vida do host na plataforma
- Usar a variável host_since
- Aqui vou trabalhar com a contagem de meses

In [16]:
# Criando variável em dias
df1["host_since"] = df1["host_since"].astype("string")
df1["host_since"] = pd.to_datetime(df1["host_since"])
df1["last_day"] = "2021-02-27"
df1["last_day"] = pd.to_datetime(df1["last_day"])
df1["host_since_d"] = df1["last_day"] - df1["host_since"]
df1["host_since_d"] = df1["host_since_d"] / np.timedelta64(1, "D")
print(pd.value_counts(df1["host_since_d"]))

143.0     298
1648.0    204
2510.0     86
2716.0     73
2530.0     69
         ... 
3488.0      1
629.0       1
3168.0      1
3070.0      1
3619.0      1
Name: host_since_d, Length: 3392, dtype: int64


In [17]:
# Criando variável em meses
df1["host_since"] = df1["host_since"].astype("string")
df1["host_since"] = pd.to_datetime(df1["host_since"])
df1["last_day"] = "2021-02-27"
df1["last_day"] = pd.to_datetime(df1["last_day"])
df1["host_since_m"] = df1["last_day"] - df1["host_since"]
df1["host_since_m"] = df1["host_since_m"] / np.timedelta64(1, "M")
print(pd.value_counts(df1["host_since_m"]))

4.698248     298
54.144849    204
82.465759     86
89.233865     73
57.036079     69
            ... 
34.497628      1
47.869566      1
11.729194      1
28.189491      1
66.892544      1
Name: host_since_m, Length: 3392, dtype: int64


In [18]:
# Criando variável em anos
df1["host_since"] = df1["host_since"].astype("string")
df1["host_since"] = pd.to_datetime(df1["host_since"])
df1["last_day"] = "2021-02-27"
df1["last_day"] = pd.to_datetime(df1["last_day"])
df1["host_since_y"] = df1["last_day"] - df1["host_since"]
df1["host_since_y"] = df1["host_since_y"] / np.timedelta64(1, "Y")
print(pd.value_counts(df1["host_since_y"]))

0.391521    298
4.512071    204
6.872147     86
7.436155     73
4.753007     69
           ... 
9.284243      1
9.683977      1
5.741391      1
3.614037      1
2.392931      1
Name: host_since_y, Length: 3392, dtype: int64


#### Identificando o país de origem do host
- Trabalhar apenas com o Brasil
- Flag se é ou não do Brasil

In [19]:
def host_verifications_bin(x):
    if x < 0:
        return 0
    else:
        return 1

In [20]:
df1["host_location_"] = df1["host_location"].str.find("Brazil")
df1["host_location_brazil"] = df1["host_location_"].apply(host_verifications_bin)
pd.value_counts(df1["host_location_brazil"])

1    20860
0     5755
Name: host_location_brazil, dtype: int64

#### Link de imagem marcado como original ou não
- Notei que existe essa disitinção nas imagens
- Criando uma variável binária, usando a variável host_thumbnail_url

In [21]:
df1["host_thumbnail_url"].head(10)

0    https://a0.muscache.com/im/pictures/user/67b13cea-8c11-49c0-a08d-7f42c330676e.jpg?aki_policy=profile_small
1    https://a0.muscache.com/im/pictures/user/6b40475c-26f6-437a-a51d-0a02e938b9e5.jpg?aki_policy=profile_small
2    https://a0.muscache.com/im/pictures/user/fffed139-d5ea-41ac-926d-eb4c4717bfbd.jpg?aki_policy=profile_small
3          https://a0.muscache.com/im/users/153232/profile_pic/1317563156/original.jpg?aki_policy=profile_small
4          https://a0.muscache.com/im/users/153691/profile_pic/1277774787/original.jpg?aki_policy=profile_small
5          https://a0.muscache.com/im/users/178975/profile_pic/1384245754/original.jpg?aki_policy=profile_small
6    https://a0.muscache.com/im/pictures/user/cee2845b-0e43-4700-8e78-c93f8893c231.jpg?aki_policy=profile_small
7          https://a0.muscache.com/im/users/222884/profile_pic/1283553621/original.jpg?aki_policy=profile_small
8          https://a0.muscache.com/im/users/224192/profile_pic/1358100631/original.jpg?aki_policy=profil

In [22]:
df1["host_thumbnail_url_"] = df1["host_thumbnail_url"].str.find("original")
pd.set_option("max_colwidth", 10)
df1.head(3)

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,last_scraped_d,last_scraped_m,last_scraped_y,picture_url_tipo,host_since_d,host_since_m,host_since_y,host_location_,host_location_brazil,host_thumbnail_url_
0,17878,https:...,202102...,2021-02-23,Very N...,Discou...,This i...,https:...,68997,https:...,...,4.0,0.13142,0.010952,jpg,4068.0,133.65...,11.137806,41.0,1,-1.0
1,24480,https:...,202102...,2021-02-23,Nice a...,My stu...,The be...,https:...,99249,https:...,...,4.0,0.13142,0.010952,jpg,3991.0,131.12...,10.926987,32.0,1,-1.0
2,25026,https:...,202102...,2021-02-23,Beauti...,Our ap...,Copaca...,https:...,3746246,https:...,...,4.0,0.13142,0.010952,jpg,3069.0,100.83...,8.402637,32.0,1,-1.0


In [23]:
def host_thumbnail_url_bin(x):
    if x < 0:
        return 0
    else:
        return 1

In [24]:
df1["host_thumbnail_url_o"] = df1["host_thumbnail_url_"].apply(host_thumbnail_url_bin)
pd.value_counts(df1["host_thumbnail_url_o"])

0    21309
1     5306
Name: host_thumbnail_url_o, dtype: int64

#### Link de imagem marcado como original ou não
- Notei que existe essa disitinção nas imagens
- Criando uma variável binária, usando a variável host_picture_url

_Obs.: Notei que é igual ao host_thumbnail_url, por isso não vou usar._

In [25]:
pd.set_option("max_colwidth", None)
df1["host_picture_url"].head()

0    https://a0.muscache.com/im/pictures/user/67b13cea-8c11-49c0-a08d-7f42c330676e.jpg?aki_policy=profile_x_medium
1    https://a0.muscache.com/im/pictures/user/6b40475c-26f6-437a-a51d-0a02e938b9e5.jpg?aki_policy=profile_x_medium
2    https://a0.muscache.com/im/pictures/user/fffed139-d5ea-41ac-926d-eb4c4717bfbd.jpg?aki_policy=profile_x_medium
3          https://a0.muscache.com/im/users/153232/profile_pic/1317563156/original.jpg?aki_policy=profile_x_medium
4          https://a0.muscache.com/im/users/153691/profile_pic/1277774787/original.jpg?aki_policy=profile_x_medium
Name: host_picture_url, dtype: object

In [26]:
df1["host_picture_url_"] = df1["host_picture_url"].str.find("original")
pd.set_option("max_colwidth", 10)
df1.head(3)

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,last_scraped_y,picture_url_tipo,host_since_d,host_since_m,host_since_y,host_location_,host_location_brazil,host_thumbnail_url_,host_thumbnail_url_o,host_picture_url_
0,17878,https:...,202102...,2021-02-23,Very N...,Discou...,This i...,https:...,68997,https:...,...,0.010952,jpg,4068.0,133.65...,11.137806,41.0,1,-1.0,0,-1.0
1,24480,https:...,202102...,2021-02-23,Nice a...,My stu...,The be...,https:...,99249,https:...,...,0.010952,jpg,3991.0,131.12...,10.926987,32.0,1,-1.0,0,-1.0
2,25026,https:...,202102...,2021-02-23,Beauti...,Our ap...,Copaca...,https:...,3746246,https:...,...,0.010952,jpg,3069.0,100.83...,8.402637,32.0,1,-1.0,0,-1.0


In [27]:
def host_picture_url_bin(x):
    if x < 0:
        return 0
    else:
        return 1

In [28]:
df1["host_picture_url_o"] = df1["host_picture_url_"].apply(host_picture_url_bin)
pd.value_counts(df1["host_picture_url_o"])

0    21309
1     5306
Name: host_picture_url_o, dtype: int64

#### Criando variáveis binárias a partir da host_verifications
- Identificar itens com maior frequência
- Criando variável binária para cada item

In [29]:
pd.set_option("max_colwidth", None)
df1["host_verifications"].head(5)

0    ['email', 'phone', 'reviews', 'jumio', 'offline_government_id', 'selfie', 'government_id']
1                           ['email', 'phone', 'facebook', 'reviews', 'jumio', 'government_id']
2                       ['phone', 'reviews', 'jumio', 'offline_government_id', 'government_id']
3         ['email', 'phone', 'facebook', 'reviews', 'manual_offline', 'jumio', 'government_id']
4                                       ['email', 'phone', 'reviews', 'jumio', 'government_id']
Name: host_verifications, dtype: object

In [30]:
df1["host_verifications_"] = df1["host_verifications"].str.replace("[,\[\]\']", "")
pd.set_option("max_colwidth", None)
df1["host_verifications_"].head(5)

0    email phone reviews jumio offline_government_id selfie government_id
1                        email phone facebook reviews jumio government_id
2                 phone reviews jumio offline_government_id government_id
3         email phone facebook reviews manual_offline jumio government_id
4                                 email phone reviews jumio government_id
Name: host_verifications_, dtype: object

In [31]:
freq = pd.Series(' '.join(df1["host_verifications_"]).split()).value_counts()[:]
print(freq)

phone                    26190
email                    23854
government_id            16853
jumio                    14263
reviews                  13235
offline_government_id    11516
selfie                    7984
identity_manual           7331
facebook                  6192
work_email                2513
google                    1454
manual_offline             570
manual_online              130
kba                        124
None                        24
sent_id                     18
dtype: int64


In [32]:
def host_verifications_bin(x):
    if x < 0:
        return 0
    else:
        return 1

In [33]:
df1["host_verifications_"] = df1["host_verifications"].str.find("phone")
df1["host_verifications_phone"] = df1["host_verifications_"].apply(host_verifications_bin)
pd.value_counts(df1["host_verifications_phone"])

1    26190
0      425
Name: host_verifications_phone, dtype: int64

In [34]:
df1["host_verifications_"] = df1["host_verifications"].str.find("email")
df1["host_verifications_email"] = df1["host_verifications_"].apply(host_verifications_bin)
pd.value_counts(df1["host_verifications_email"])

1    23948
0     2667
Name: host_verifications_email, dtype: int64

In [35]:
df1["host_verifications_"] = df1["host_verifications"].str.find("government_id")
df1["host_verifications_government_id"] = df1["host_verifications_"].apply(host_verifications_bin)
pd.value_counts(df1["host_verifications_government_id"])

1    16855
0     9760
Name: host_verifications_government_id, dtype: int64

In [36]:
df1["host_verifications_"] = df1["host_verifications"].str.find("jumio")
df1["host_verifications_jumio"] = df1["host_verifications_"].apply(host_verifications_bin)
pd.value_counts(df1["host_verifications_jumio"])

1    14263
0    12352
Name: host_verifications_jumio, dtype: int64

In [37]:
df1["host_verifications_"] = df1["host_verifications"].str.find("reviews")
df1["host_verifications_reviews"] = df1["host_verifications_"].apply(host_verifications_bin)
pd.value_counts(df1["host_verifications_reviews"])

0    13380
1    13235
Name: host_verifications_reviews, dtype: int64

In [38]:
df1["host_verifications_"] = df1["host_verifications"].str.find("offline_government_id")
df1["host_verifications_offline_government_id"] = df1["host_verifications_"].apply(host_verifications_bin)
pd.value_counts(df1["host_verifications_offline_government_id"])

0    15099
1    11516
Name: host_verifications_offline_government_id, dtype: int64

In [39]:
df1["host_verifications_"] = df1["host_verifications"].str.find("selfie")
df1["host_verifications_selfie"] = df1["host_verifications_"].apply(host_verifications_bin)
pd.value_counts(df1["host_verifications_selfie"])

0    18631
1     7984
Name: host_verifications_selfie, dtype: int64

In [40]:
df1["host_verifications_"] = df1["host_verifications"].str.find("identity_manual")
df1["host_verifications_identity_manual"] = df1["host_verifications_"].apply(host_verifications_bin)
pd.value_counts(df1["host_verifications_identity_manual"])

0    19284
1     7331
Name: host_verifications_identity_manual, dtype: int64

In [41]:
df1["host_verifications_"] = df1["host_verifications"].str.find("facebook")
df1["host_verifications_facebook"] = df1["host_verifications_"].apply(host_verifications_bin)
pd.value_counts(df1["host_verifications_facebook"])

0    20423
1     6192
Name: host_verifications_facebook, dtype: int64

In [42]:
df1["host_verifications_"] = df1["host_verifications"].str.find("work_email")
df1["host_verifications_work_email"] = df1["host_verifications_"].apply(host_verifications_bin)
pd.value_counts(df1["host_verifications_work_email"])

0    24102
1     2513
Name: host_verifications_work_email, dtype: int64

In [43]:
df1["host_verifications_"] = df1["host_verifications"].str.find("google")
df1["host_verifications_google"] = df1["host_verifications_"].apply(host_verifications_bin)
pd.value_counts(df1["host_verifications_google"])

0    25161
1     1454
Name: host_verifications_google, dtype: int64

#### Quanditade de banheiros
- Removendo parte numérica da variável bathrooms_text
- Transformando em float64

In [44]:
bathrooms_text = pd.DataFrame(df1.bathrooms_text.str.split(" ", expand = True))
print(bathrooms_text.head(5))

     0      1     2
0    1   bath  None
1    1   bath  None
2    1   bath  None
3  1.5  baths  None
4    1   bath  None


In [45]:
bathrooms_text = pd.DataFrame({"bathrooms_text_vol": bathrooms_text.iloc[:,0]})
bathrooms_text.head()

,bathrooms_text_vol
0,1
1,1
2,1
3,1.5
4,1


In [46]:
pd.value_counts(bathrooms_text["bathrooms_text_vol"])

1            14645
2             5870
1.5           1738
3             1641
2.5           1040
3.5            429
4              388
4.5            176
5              153
6               93
0               78
5.5             77
7               63
6.5             42
8               41
7.5             16
Shared          13
Half-bath        8
11               5
8.5              4
Private          4
9                3
20               2
10               2
10.5             2
15               2
12               2
9.5              1
13               1
12.5             1
11.5             1
15.5             1
18               1
Name: bathrooms_text_vol, dtype: int64

In [47]:
bathrooms_text["bathrooms_text_vol"] = bathrooms_text["bathrooms_text_vol"].str.replace("Shared","1" )
bathrooms_text["bathrooms_text_vol"] = bathrooms_text["bathrooms_text_vol"].str.replace("Private","1" )
bathrooms_text["bathrooms_text_vol"] = bathrooms_text["bathrooms_text_vol"].str.replace("Half-bath","1" )
bathrooms_text["bathrooms_text_vol"] = bathrooms_text["bathrooms_text_vol"].astype("float64")
df1["bathrooms_text_vol"] = bathrooms_text["bathrooms_text_vol"]
pd.value_counts(df1["bathrooms_text_vol"])

1.0     14670
2.0      5870
1.5      1738
3.0      1641
2.5      1040
3.5       429
4.0       388
4.5       176
5.0       153
6.0        93
0.0        78
5.5        77
7.0        63
6.5        42
8.0        41
7.5        16
11.0        5
8.5         4
9.0         3
12.0        2
10.0        2
20.0        2
10.5        2
15.0        2
12.5        1
18.0        1
9.5         1
15.5        1
13.0        1
11.5        1
Name: bathrooms_text_vol, dtype: int64

#### Tranformando a variável host_response_rate em float
- Vou remover o %
- Vou dividir por 100

In [48]:
df1["host_response_rate_"] = df1["host_response_rate"].str.replace("%","" )
df1["host_response_rate_"] = df1["host_response_rate_"].astype("float")
df1["host_response_rate_"] = df1["host_response_rate_"] / 100
df1["host_response_rate_"].head()

0    1.00
1    1.00
2    0.97
3     NaN
4    1.00
Name: host_response_rate_, dtype: float64

#### Tranformando a variável host_acceptance_rate em float
- Vou remover o %
- Vou dividir por 100

In [49]:
df1["host_acceptance_rate_"] = df1["host_acceptance_rate"].str.replace("%","" )
df1["host_acceptance_rate_"] = df1["host_acceptance_rate_"].str.replace(" ","" )
df1["host_acceptance_rate_"] = df1["host_acceptance_rate_"].astype("float")
df1["host_acceptance_rate_"] = df1["host_acceptance_rate_"] / 100
df1["host_acceptance_rate_"].head()

0    1.00
1     NaN
2    0.78
3    1.00
4    0.94
Name: host_acceptance_rate_, dtype: float64

#### Tranformando a variável price em float
- Vou remover o $
- Remover as vírgulas
- Converter em numerico

In [50]:
df1["price"].head()

0    $221.00
1    $307.00
2    $160.00
3    $273.00
4    $135.00
Name: price, dtype: object

In [51]:
df1["price"].to_csv("price.csv" ,index=False)

In [52]:
df1["price_"] = df1["price"].str.replace(r'$',"")
df1["price_"] = df1["price_"].str.replace(",","" )
df1["price_"] = df1["price_"].astype("float")
df1["price_"].head(10)

0     221.0
1     307.0
2     160.0
3     273.0
4     135.0
5     484.0
6    1751.0
7     540.0
8     160.0
9     782.0
Name: price_, dtype: float64

#### Criando variável first_review para D, M e Y
- Volume de dias da data de carregamento destas bases e ultipo scraping no anúncio
- Utilizando a variável first_review para o cálculo

In [53]:
# Criando variável em dias
df1["first_review"] = df1["first_review"].astype("string")
df1["first_review"] = pd.to_datetime(df1["first_review"])
df1["last_day"] = "2021-02-27"
df1["last_day"] = pd.to_datetime(df1["last_day"])
df1["first_review_d"] = df1["last_day"] - df1["first_review"]
df1["first_review_d"] = df1["first_review_d"] / np.timedelta64(1, "D")
print(pd.value_counts(df1["first_review_d"]))

367.0     262
510.0     183
422.0     181
55.0      176
517.0     171
         ... 
688.0       1
2297.0      1
3614.0      1
1983.0      1
2816.0      1
Name: first_review_d, Length: 2610, dtype: int64


In [54]:
# Criando variável em meses
df1["first_review"] = df1["first_review"].astype("string")
df1["first_review"] = pd.to_datetime(df1["first_review"])
df1["last_day"] = "2021-02-27"
df1["last_day"] = pd.to_datetime(df1["last_day"])
df1["first_review_m"] = df1["last_day"] - df1["first_review"]
df1["first_review_m"] = df1["first_review_m"] / np.timedelta64(1, "M")
print(pd.value_counts(df1["first_review_m"]))

12.057742    262
16.755991    183
13.864761    181
1.807019     176
16.985975    171
            ... 
39.622990      1
50.826506      1
8.377995       1
83.090002      1
62.884248      1
Name: first_review_m, Length: 2610, dtype: int64


In [55]:
# Criando variável em anos
df1["first_review"] = df1["first_review"].astype("string")
df1["first_review"] = pd.to_datetime(df1["first_review"])
df1["last_day"] = "2021-02-27"
df1["last_day"] = pd.to_datetime(df1["last_day"])
df1["first_review_y"] = df1["last_day"] - df1["first_review"]
df1["first_review_y"] = df1["first_review_y"] / np.timedelta64(1, "Y")
print(pd.value_counts(df1["first_review_y"]))

1.004812    262
1.396333    183
1.155397    181
0.150585    176
1.415498    171
           ... 
5.850907      1
6.231476      1
9.139134      1
4.418982      1
5.872811      1
Name: first_review_y, Length: 2610, dtype: int64


#### Criando variável last_review para D, M e Y
- Volume de dias da data de carregamento destas bases e ultipo scraping no anúncio
- Utilizando a variável last_review para o cálculo

In [56]:
# Criando variável em dias
df1["last_review"] = df1["last_review"].astype("string")
df1["last_review"] = pd.to_datetime(df1["last_review"])
df1["last_day"] = "2021-02-27"
df1["last_day"] = pd.to_datetime(df1["last_day"])
df1["last_review_d"] = df1["last_day"] - df1["last_review"]
df1["last_review_d"] = df1["last_review_d"] / np.timedelta64(1, "D")
print(pd.value_counts(df1["last_review_d"]))

367.0     681
55.0      530
10.0      511
368.0     390
11.0      317
         ... 
574.0       1
189.0       1
234.0       1
1300.0      1
241.0       1
Name: last_review_d, Length: 1469, dtype: int64


In [57]:
# Criando variável em meses
df1["last_review"] = df1["last_review"].astype("string")
df1["last_review"] = pd.to_datetime(df1["last_review"])
df1["last_day"] = "2021-02-27"
df1["last_day"] = pd.to_datetime(df1["last_day"])
df1["last_review_m"] = df1["last_day"] - df1["last_review"]
df1["last_review_m"] = df1["last_review_m"] / np.timedelta64(1, "M")
print(pd.value_counts(df1["last_review_m"]))

12.057742    681
1.807019     530
0.328549     511
12.090597    390
0.361404     317
            ... 
21.552804      1
8.279431       1
32.296351      1
9.823610       1
77.373252      1
Name: last_review_m, Length: 1469, dtype: int64


In [58]:
# Criando variável em anos
df1["last_review"] = df1["last_review"].astype("string")
df1["last_review"] = pd.to_datetime(df1["last_review"])
df1["last_day"] = "2021-02-27"
df1["last_day"] = pd.to_datetime(df1["last_day"])
df1["last_review_y"] = df1["last_day"] - df1["last_review"]
df1["last_review_y"] = df1["last_review_y"] / np.timedelta64(1, "Y")
print(pd.value_counts(df1["last_review_y"]))

1.004812    681
0.150585    530
0.027379    511
1.007550    390
0.030117    317
           ... 
3.967227      1
2.795403      1
2.253297      1
2.978843      1
2.631129      1
Name: last_review_y, Length: 1469, dtype: int64


#### Montando dataset para rodar modelos de classificação

In [59]:
df2 = df1[["last_scraped_d",
"last_scraped_m",
"last_scraped_y",
"host_since_d",
"host_since_m",
"host_since_y",
"first_review_d",
"first_review_m",
"first_review_y",
"last_review_d",
"last_review_m",
"last_review_y",
"host_listings_count",
"host_total_listings_count",
"latitude",
"longitude",
"bathrooms_text_vol",
"bedrooms",
"minimum_minimum_nights",
"maximum_minimum_nights",
"minimum_maximum_nights",
"maximum_maximum_nights",
"minimum_nights_avg_ntm",
"maximum_nights_avg_ntm",
"review_scores_rating",
"review_scores_accuracy",
"review_scores_cleanliness",
"review_scores_checkin",
"review_scores_communication",
"review_scores_location",
"review_scores_value",
"reviews_per_month",
"host_response_rate_",
"host_acceptance_rate_",
"price_",
"host_location_brazil",
"host_thumbnail_url_o",
"host_verifications_phone",
"host_verifications_email",
"host_verifications_government_id",
"host_verifications_jumio",
"host_verifications_reviews",
"host_verifications_offline_government_id",
"host_verifications_selfie",
"host_verifications_identity_manual",
"host_verifications_facebook",
"host_verifications_work_email",
"host_verifications_google",
"accommodates",
"minimum_nights",
"maximum_nights",
"availability_30",
"availability_60",
"availability_90",
"availability_365",
"number_of_reviews",
"number_of_reviews_ltm",
"number_of_reviews_l30d",
"calculated_host_listings_count",
"calculated_host_listings_count_entire_homes",
"calculated_host_listings_count_private_rooms",
"calculated_host_listings_count_shared_rooms",
"host_response_time",
"host_is_superhost",
"host_has_profile_pic",
"host_identity_verified",
"has_availability",
"instant_bookable",
"picture_url_tipo",
"room_type"]]

In [60]:
pd.set_option("max_colwidth",10)
pd.set_option("max_rows",None)
df2.head()

,last_scraped_d,last_scraped_m,last_scraped_y,host_since_d,host_since_m,host_since_y,first_review_d,first_review_m,first_review_y,last_review_d,...,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,host_response_time,host_is_superhost,host_has_profile_pic,host_identity_verified,has_availability,instant_bookable,picture_url_tipo,room_type
0,4.0,0.131420,0.010952,4068.0,133.65...,11.137806,3880.0,127.47...,10.623079,19.0,...,0,0,within...,t,t,t,t,t,jpg,Entire...
1,4.0,0.131420,0.010952,3991.0,131.12...,10.926987,3807.0,125.07...,10.423212,1109.0,...,0,0,within...,f,t,t,t,f,jpg,Entire...
2,4.0,0.131420,0.010952,3069.0,100.83...,8.402637,3918.0,128.72...,10.727120,378.0,...,0,0,within...,f,t,f,t,f,jpg,Entire...
3,3.0,0.098565,0.008214,3898.0,128.06...,10.672362,2685.0,88.215364,7.351280,349.0,...,0,0,NaN,t,t,t,t,f,jpg,Entire...
4,4.0,0.131420,0.010952,3898.0,128.06...,10.672362,3800.0,124.84...,10.404047,17.0,...,0,0,within...,t,t,t,t,f,jpg,Entire...


In [61]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26615 entries, 0 to 26614
Data columns (total 70 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   last_scraped_d                                26615 non-null  float64
 1   last_scraped_m                                26615 non-null  float64
 2   last_scraped_y                                26615 non-null  float64
 3   host_since_d                                  26591 non-null  float64
 4   host_since_m                                  26591 non-null  float64
 5   host_since_y                                  26591 non-null  float64
 6   first_review_d                                16657 non-null  float64
 7   first_review_m                                16657 non-null  float64
 8   first_review_y                                16657 non-null  float64
 9   last_review_d                                 16657 non-null 

#### Tratando missing values nas variáveis float
- Vou substituir pela média da coluna, em variávis quantitativas
- Vou substituir pelo valor mais frequênte da coluna, em variávis qualitativas

In [62]:
df2.isnull().sum()

last_scraped_d                                      0
last_scraped_m                                      0
last_scraped_y                                      0
host_since_d                                       24
host_since_m                                       24
host_since_y                                       24
first_review_d                                   9958
first_review_m                                   9958
first_review_y                                   9958
last_review_d                                    9958
last_review_m                                    9958
last_review_y                                    9958
host_listings_count                                24
host_total_listings_count                          24
latitude                                            0
longitude                                           0
bathrooms_text_vol                                 72
bedrooms                                         1746
minimum_minimum_nights      

In [63]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(df2.iloc[:, 0:35])
df2.iloc[:, 0:35] = imputer.transform(df2.iloc[:, 0:35])

In [64]:
df2.isnull().sum()

last_scraped_d                                     0
last_scraped_m                                     0
last_scraped_y                                     0
host_since_d                                       0
host_since_m                                       0
host_since_y                                       0
first_review_d                                     0
first_review_m                                     0
first_review_y                                     0
last_review_d                                      0
last_review_m                                      0
last_review_y                                      0
host_listings_count                                0
host_total_listings_count                          0
latitude                                           0
longitude                                          0
bathrooms_text_vol                                 0
bedrooms                                           0
minimum_minimum_nights                        

In [65]:
df2.iloc[:, 62:70] = df2.iloc[:, 62:70].astype("category")

In [66]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer = imputer.fit(df2.iloc[:, 62:70])
df2.iloc[:, 62:70] = imputer.transform(df2.iloc[:, 62:70])

In [67]:
df2.isnull().sum()

last_scraped_d                                  0
last_scraped_m                                  0
last_scraped_y                                  0
host_since_d                                    0
host_since_m                                    0
host_since_y                                    0
first_review_d                                  0
first_review_m                                  0
first_review_y                                  0
last_review_d                                   0
last_review_m                                   0
last_review_y                                   0
host_listings_count                             0
host_total_listings_count                       0
latitude                                        0
longitude                                       0
bathrooms_text_vol                              0
bedrooms                                        0
minimum_minimum_nights                          0
maximum_minimum_nights                          0


#### Categorizando as variáveis com números
- Utilizando o label encoder

In [68]:
host_response_time_l = df2["host_response_time"].values
host_is_superhost_l = df2["host_is_superhost"].values
host_has_profile_pic_l = df2["host_has_profile_pic"].values
host_identity_verified_l = df2["host_identity_verified"].values
has_availability_l = df2["has_availability"].values
instant_bookable_l = df2["instant_bookable"].values
picture_url_tipo_l = df2["picture_url_tipo"].values
room_type_l = df2["room_type"].values

In [69]:
labelencoder_df2 = LabelEncoder()
host_response_time_l = pd.DataFrame({"host_response_time":labelencoder_df2.fit_transform(host_response_time_l)})
host_is_superhost_l = pd.DataFrame({"host_is_superhost":labelencoder_df2.fit_transform(host_is_superhost_l)})
host_has_profile_pic_l = pd.DataFrame({"host_has_profile_pic": labelencoder_df2.fit_transform(host_has_profile_pic_l)})
host_identity_verified_l = pd.DataFrame({"host_identity_verified":labelencoder_df2.fit_transform(host_identity_verified_l)})
has_availability_l = pd.DataFrame({"has_availability":labelencoder_df2.fit_transform(has_availability_l)})
instant_bookable_l = pd.DataFrame({"instant_bookable":labelencoder_df2.fit_transform(instant_bookable_l)})
picture_url_tipo_l = pd.DataFrame({"picture_url_tipo":labelencoder_df2.fit_transform(picture_url_tipo_l)})
room_type_l = pd.DataFrame({"room_type":labelencoder_df2.fit_transform(room_type_l)})

In [70]:
df2["host_response_time"] = host_response_time_l["host_response_time"]
df2["host_is_superhost"] = host_is_superhost_l["host_is_superhost"]
df2["host_has_profile_pic"] = host_has_profile_pic_l["host_has_profile_pic"]
df2["host_identity_verified"] = host_identity_verified_l["host_identity_verified"]
df2["has_availability"] = has_availability_l["has_availability"]
df2["instant_bookable"] = instant_bookable_l["instant_bookable"]
df2["picture_url_tipo"] = picture_url_tipo_l["picture_url_tipo"]
df2["room_type"] = room_type_l["room_type"]

In [71]:
df2.head()

,last_scraped_d,last_scraped_m,last_scraped_y,host_since_d,host_since_m,host_since_y,first_review_d,first_review_m,first_review_y,last_review_d,...,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,host_response_time,host_is_superhost,host_has_profile_pic,host_identity_verified,has_availability,instant_bookable,picture_url_tipo,room_type
0,4.0,0.131420,0.010952,4068.0,133.65...,11.137806,3880.0,127.47...,10.623079,19.0,...,0,0,3,1,1,1,1,1,1,0
1,4.0,0.131420,0.010952,3991.0,131.12...,10.926987,3807.0,125.07...,10.423212,1109.0,...,0,0,2,0,1,1,1,0,1,0
2,4.0,0.131420,0.010952,3069.0,100.83...,8.402637,3918.0,128.72...,10.727120,378.0,...,0,0,2,0,1,0,1,0,1,0
3,3.0,0.098565,0.008214,3898.0,128.06...,10.672362,2685.0,88.215364,7.351280,349.0,...,0,0,3,1,1,1,1,0,1,0
4,4.0,0.131420,0.010952,3898.0,128.06...,10.672362,3800.0,124.84...,10.404047,17.0,...,0,0,3,1,1,1,1,0,1,0


In [72]:
df2 = df2[["accommodates",
"availability_30",
"availability_365",
"availability_60",
"availability_90",
"bathrooms_text_vol",
"bedrooms",
"calculated_host_listings_count",
"calculated_host_listings_count_entire_homes",
"calculated_host_listings_count_private_rooms",
"calculated_host_listings_count_shared_rooms",
"first_review_d",
"first_review_m",
"first_review_y",
"host_acceptance_rate_",
"host_listings_count",
"host_response_rate_",
"host_response_time",
"host_since_d",
"host_since_m",
"host_since_y",
"host_total_listings_count",
"last_review_d",
"last_review_m",
"last_review_y",
"last_scraped_d",
"last_scraped_m",
"last_scraped_y",
"latitude",
"longitude",
"maximum_maximum_nights",
"maximum_minimum_nights",
"maximum_nights",
"maximum_nights_avg_ntm",
"minimum_maximum_nights",
"minimum_minimum_nights",
"minimum_nights",
"minimum_nights_avg_ntm",
"number_of_reviews",
"number_of_reviews_l30d",
"number_of_reviews_ltm",
"price_",
"review_scores_accuracy",
"review_scores_checkin",
"review_scores_cleanliness",
"review_scores_communication",
"review_scores_location",
"review_scores_rating",
"review_scores_value",
"reviews_per_month",
"has_availability",
"host_has_profile_pic",
"host_identity_verified",
"host_is_superhost",
"host_location_brazil",
"host_thumbnail_url_o",
"host_verifications_email",
"host_verifications_facebook",
"host_verifications_google",
"host_verifications_government_id",
"host_verifications_identity_manual",
"host_verifications_jumio",
"host_verifications_offline_government_id",
"host_verifications_phone",
"host_verifications_reviews",
"host_verifications_selfie",
"host_verifications_work_email",
"instant_bookable",
"picture_url_tipo",
"room_type",]]

In [73]:
df2.columns

Index(['accommodates', 'availability_30', 'availability_365',
       'availability_60', 'availability_90', 'bathrooms_text_vol', 'bedrooms',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'first_review_d',
       'first_review_m', 'first_review_y', 'host_acceptance_rate_',
       'host_listings_count', 'host_response_rate_', 'host_response_time',
       'host_since_d', 'host_since_m', 'host_since_y',
       'host_total_listings_count', 'last_review_d', 'last_review_m',
       'last_review_y', 'last_scraped_d', 'last_scraped_m', 'last_scraped_y',
       'latitude', 'longitude', 'maximum_maximum_nights',
       'maximum_minimum_nights', 'maximum_nights', 'maximum_nights_avg_ntm',
       'minimum_maximum_nights', 'minimum_minimum_nights', 'minimum_nights',
       'minimum_nights_avg_ntm', 'number_of_reviews', 'number_of_reviews_l30d',

#### Separando as bases
- X = Variáveis indendentes
- y = Variável dependente

In [74]:
X = df2.drop(['room_type'], axis=1)
y = df2['room_type']

In [75]:
X.head()

,accommodates,availability_30,availability_365,availability_60,availability_90,bathrooms_text_vol,bedrooms,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,...,host_verifications_government_id,host_verifications_identity_manual,host_verifications_jumio,host_verifications_offline_government_id,host_verifications_phone,host_verifications_reviews,host_verifications_selfie,host_verifications_work_email,instant_bookable,picture_url_tipo
0,5,0,304,8,38,1.0,2.0,1,1,0,...,1,0,1,1,1,1,1,0,1,1
1,2,0,10,0,0,1.0,1.0,1,1,0,...,1,0,1,0,1,1,0,0,0,1
2,2,0,328,23,53,1.0,1.0,11,11,0,...,1,0,1,1,1,1,0,0,0,1
3,2,0,207,0,0,1.5,1.0,1,1,0,...,1,0,1,0,1,1,0,0,0,1
4,2,4,101,4,22,1.0,1.0,1,1,0,...,1,0,1,0,1,1,0,0,0,1


In [76]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: room_type, dtype: int32

In [77]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26615 entries, 0 to 26614
Data columns (total 69 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   accommodates                                  26615 non-null  int64  
 1   availability_30                               26615 non-null  int64  
 2   availability_365                              26615 non-null  int64  
 3   availability_60                               26615 non-null  int64  
 4   availability_90                               26615 non-null  int64  
 5   bathrooms_text_vol                            26615 non-null  float64
 6   bedrooms                                      26615 non-null  float64
 7   calculated_host_listings_count                26615 non-null  int64  
 8   calculated_host_listings_count_entire_homes   26615 non-null  int64  
 9   calculated_host_listings_count_private_rooms  26615 non-null 

#### Selecionando as variáveis categóricas

In [78]:
base = X[["has_availability",
"host_has_profile_pic",
"host_identity_verified",
"host_is_superhost",
"host_location_brazil",
"host_thumbnail_url_o",
"host_verifications_email",
"host_verifications_facebook",
"host_verifications_google",
"host_verifications_government_id",
"host_verifications_identity_manual",
"host_verifications_jumio",
"host_verifications_offline_government_id",
"host_verifications_phone",
"host_verifications_reviews",
"host_verifications_selfie",
"host_verifications_work_email",
"instant_bookable",
"picture_url_tipo"]]

test = SelectKBest(chi2, k=10)
fit = test.fit(base, y)
features = fit.transform(base)

In [79]:
fit.get_support(indices=True)

array([ 2,  3,  9, 10, 11, 12, 14, 15, 16, 17], dtype=int64)

In [80]:
cols = fit.get_support(indices=True)
base_ = base.iloc[:,cols]
base_.columns

Index(['host_identity_verified', 'host_is_superhost',
       'host_verifications_government_id',
       'host_verifications_identity_manual', 'host_verifications_jumio',
       'host_verifications_offline_government_id',
       'host_verifications_reviews', 'host_verifications_selfie',
       'host_verifications_work_email', 'instant_bookable'],
      dtype='object')

#### Selecionando as variáveis quantitativas

In [81]:
base = X[["accommodates",
"availability_30",
"availability_365",
"availability_60",
"availability_90",
"bathrooms_text_vol",
"bedrooms",
"calculated_host_listings_count",
"calculated_host_listings_count_entire_homes",
"calculated_host_listings_count_private_rooms",
"calculated_host_listings_count_shared_rooms",
"first_review_d",
"first_review_m",
"first_review_y",
"host_acceptance_rate_",
"host_listings_count",
"host_response_rate_",
"host_response_time",
"host_since_d",
"host_since_m",
"host_since_y",
"host_total_listings_count",
"last_review_d",
"last_review_m",
"last_review_y",
"last_scraped_d",
"last_scraped_m",
"last_scraped_y",
"latitude",
"longitude",
"maximum_maximum_nights",
"maximum_minimum_nights",
"maximum_nights",
"maximum_nights_avg_ntm",
"minimum_maximum_nights",
"minimum_minimum_nights",
"minimum_nights",
"minimum_nights_avg_ntm",
"number_of_reviews",
"number_of_reviews_l30d",
"number_of_reviews_ltm",
"price_",
"review_scores_accuracy",
"review_scores_checkin",
"review_scores_cleanliness",
"review_scores_communication",
"review_scores_location",
"review_scores_rating",
"review_scores_value",
"reviews_per_month"]]

model = RandomForestClassifier(n_estimators=10)
model.fit(base, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [82]:
feature_importances = pd.DataFrame(model.feature_importances_,
                                   index = base.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)
feature_importances

,importance
calculated_host_listings_count_private_rooms,0.258095
calculated_host_listings_count_entire_homes,0.238204
accommodates,0.091112
bedrooms,0.071146
calculated_host_listings_count_shared_rooms,0.049351
price_,0.037344
calculated_host_listings_count,0.021956
minimum_nights_avg_ntm,0.019421
maximum_minimum_nights,0.015439
latitude,0.014183


In [83]:
feature_importances1 = feature_importances[feature_importances["importance"] > 0.01]
feature_importances1

,importance
calculated_host_listings_count_private_rooms,0.258095
calculated_host_listings_count_entire_homes,0.238204
accommodates,0.091112
bedrooms,0.071146
calculated_host_listings_count_shared_rooms,0.049351
price_,0.037344
calculated_host_listings_count,0.021956
minimum_nights_avg_ntm,0.019421
maximum_minimum_nights,0.015439
latitude,0.014183


#### Ajustando a base X com variáveis escolhidas
- Vou usar todas das categóricas, escolhidas pelo SelectKBest
- Vou usar todas quantitativas, acima de 0,01 de importância (de acorod com o feature importances)

In [84]:
X1 = X[['host_identity_verified', 'host_is_superhost',
       'host_verifications_government_id',
       'host_verifications_identity_manual', 'host_verifications_jumio',
       'host_verifications_offline_government_id',
       'host_verifications_reviews', 'host_verifications_selfie',
       'host_verifications_work_email', 'instant_bookable',
        
        "calculated_host_listings_count_private_rooms",
        "calculated_host_listings_count_entire_homes",
        "accommodates",
        "bedrooms",
        "calculated_host_listings_count_shared_rooms",
        "price_",
        "calculated_host_listings_count",
        "minimum_minimum_nights",
        "host_total_listings_count",
        "latitude",
        "minimum_nights",
        "last_review_d"]]

In [85]:
X1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26615 entries, 0 to 26614
Data columns (total 22 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   host_identity_verified                        26615 non-null  int32  
 1   host_is_superhost                             26615 non-null  int32  
 2   host_verifications_government_id              26615 non-null  int64  
 3   host_verifications_identity_manual            26615 non-null  int64  
 4   host_verifications_jumio                      26615 non-null  int64  
 5   host_verifications_offline_government_id      26615 non-null  int64  
 6   host_verifications_reviews                    26615 non-null  int64  
 7   host_verifications_selfie                     26615 non-null  int64  
 8   host_verifications_work_email                 26615 non-null  int64  
 9   instant_bookable                              26615 non-null 

#### Criando variáveis binárias com as variáveis com mais de uma categoria
- Utilizando o OneHotEncoder

In [91]:
X1_ = X1.iloc[:,0:10].values

onehotencoder = OneHotEncoder(sparse=False)
X1_ = onehotencoder.fit_transform(X1_)

In [87]:
X1_ = pd.DataFrame(X1_)
X2 = pd.concat([X1_, X1.iloc[:,10:19]], axis=1)
X2 = X2.values

#### Normalizando os dados

In [88]:
ss = StandardScaler()
X3 = ss.fit_transform(X2)

In [89]:
X3

array([[-0.76496388,  0.76496388, -2.20127152, ..., -0.24109808,
         0.01798324, -0.07915475],
       [-0.76496388,  0.76496388,  0.4542829 , ..., -0.24109808,
        -0.08795952, -0.1006421 ],
       [ 1.30725126, -1.30725126,  0.4542829 , ...,  0.03713222,
         0.123926  ,  0.01753832],
       ...,
       [-0.76496388,  0.76496388,  0.4542829 , ..., -0.24109808,
        -0.08795952, -0.1006421 ],
       [-0.76496388,  0.76496388, -2.20127152, ..., -0.18545202,
         0.01798324, -0.06841108],
       [-0.76496388,  0.76496388,  0.4542829 , ...,  1.37263763,
        -0.19390229,  0.55472206]])

#### Dividindo grupos de treino e teste

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X3, y, test_size = 0.3, random_state = 0)

#### Aplicando modelo de SVM

In [175]:
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [176]:
y_pred = classifier.predict(X_test)

In [177]:
def Accuracy(classifier,X3, y):
    results = cross_val_predict(classifier, X3, y, cv = 10)
    return accuracy_score(y, results)

In [ ]:
Accuracy(classifier, X3, y)

In [ ]:
results = cross_val_predict(classifier, X3, y, cv = 10)
class_values = [0,1,2,3]
print(classification_report(y, results, class_values))

#### Aplicando modelo de Árvore de Decisão

In [135]:
classifier2 = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier2.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [136]:
y_pred2 = classifier2.predict(X_test)

In [137]:
def Accuracy2(classifier2,X3, y):
    results2 = cross_val_predict(classifier2, X3, y, cv = 10)
    return accuracy_score(y, results2)

In [138]:
Accuracy2(classifier2, X3, y)

0.9616006011647568

In [139]:
results2 = cross_val_predict(classifier2, X3, y, cv = 10)
class_values = [0,1,2,3]
print(classification_report(y, results2, class_values))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98     19285
           1       0.57      0.61      0.59        99
           2       0.94      0.92      0.93      6619
           3       0.84      0.85      0.85       612

    accuracy                           0.96     26615
   macro avg       0.83      0.84      0.84     26615
weighted avg       0.96      0.96      0.96     26615



#### Aplicando modelo de Naive Bayes

In [657]:
classifier3 = GaussianNB()
classifier3.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [658]:
y_pred3 = classifier3.predict(X_test)

In [659]:
def Accuracy3(classifier3,X3, y):
    results3 = cross_val_predict(classifier3, X3, y, cv = 10)
    return accuracy_score(y, results3)

In [660]:
Accuracy3(classifier3, X3, y)

0.3632538042457261

In [661]:
results3 = cross_val_predict(classifier3, X3, y, cv = 10)
class_values = [0,1,2,3]
print(classification_report(y, results3, class_values))

              precision    recall  f1-score   support

           0       0.93      0.45      0.61     19285
           1       0.01      0.25      0.01        99
           2       0.14      0.06      0.08      6619
           3       0.06      0.97      0.11       612

    accuracy                           0.36     26615
   macro avg       0.28      0.43      0.20     26615
weighted avg       0.71      0.36      0.46     26615



#### Aplicando modelo de Random Forest

In [662]:
classifier4 = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier4.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [663]:
y_pred4 = classifier4.predict(X_test)

In [664]:
def Accuracy4(classifier4,X3, y):
    results4 = cross_val_predict(classifier4, X3, y, cv = 10)
    return accuracy_score(y, results4)

In [665]:
Accuracy4(classifier4, X3, y)

0.9676122487319181

In [666]:
results4 = cross_val_predict(classifier4, X3, y, cv = 10)
class_values = [0,1,2,3]
print(classification_report(y, results4, class_values))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98     19285
           1       0.76      0.52      0.61        99
           2       0.94      0.93      0.94      6619
           3       0.87      0.79      0.83       612

    accuracy                           0.97     26615
   macro avg       0.89      0.81      0.84     26615
weighted avg       0.97      0.97      0.97     26615



#### Aplicando modelo de XGBoost

In [93]:
classifier5 = XGBClassifier()
classifier5.fit(X_train, y_train)

[23:24:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [95]:
y_pred5 = classifier5.predict(X_test)

In [154]:
def Accuracy5(classifier5,X3, y):
    results5 = cross_val_predict(classifier5, X3, y, cv = 10)
    return accuracy_score(y, results5)

In [155]:
Accuracy5(classifier5, X3, y)

[22:18:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:18:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:18:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:18:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

0.9701671989479617

In [99]:
results5 = cross_val_predict(classifier5, X3, y, cv = 10)
class_values = [0,1,2,3]
print(classification_report(y, results5, class_values))

[23:31:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:31:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:31:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:31:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

In [96]:
print(pd.crosstab(y_test, y_pred5,
                 rownames=['Real'], colnames=['Predito'],
                 margins=True))

Predito     0   1     2    3   All
Real                              
0        5654   2    69    5  5730
1           2  17     5    1    25
2         101   1  1929   22  2053
3           1   0     8  168   177
All      5758  20  2011  196  7985
